# Adversarial Attacks

The following code is loading the Tesseract dataset towards doing an adversarial attack.

In [ ]:
# execute only once to add home directory
import sys
sys.path.insert(0,'../')

# This will integrate functions to load the datasets. 
# There are four functions: one for X, one for y, and one for metadata associated with the samples, and one for time-index.
import libs.utils

# Important: change this parameter to the folder where you inserted the dataset (see README.md in lab 02)
dataset_path = '../datasets/drebin/'

In [ ]:
import os
import json
import datetime
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split

X, feature_names = libs.utils.load_X(dataset_path, reduced=True)
y = libs.utils.load_y(dataset_path)

# Following DREBIN feature-space
clf = LinearSVC(C=1, dual=True, max_iter=10000)

# Random split of train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the classifier
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

## Exercise 1: Evasion of a single point (Linear Classifier)

Generate an adversarial attack for a single point. Get the most important benign feature of the svm (hint: look at `coef_[0]`), and swap it to a 1.0 (the feature space is binary).

Notes: you should attack only true positives!

In [ ]:
# Attacking a single point
tp_mask = (y_pred==1.0) & (y_test==1.0)
X_tps = X_test[tp_mask,:]

test_point = X_tps[0].toarray()
print('Before attack: {}'.format(clf.predict(test_point)))
top_10_benign_idx = np.argsort(clf.coef_[0])[:10]
test_point[:, top_10_benign_idx] = 1.0
print('After attack: {}'.format(clf.predict(test_point)))

## Exercise 2: Security Evaluation Curves (Linear Classifier)

Create a function that modifies the top-1, top-2, ... top-N features to all samples, and draws a security evaluation curve as follows: the number of modified features on the X-axis, and the attack success rate on the Y-axis.

Note: you should attack only true positives!